In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
from datetime import date, datetime
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.api import VAR

sns.set_style("darkgrid")

In [2]:
tickers = ["CL=F", "NG=F", "RB=F"]
interval = "1d"
start_date = datetime.strptime("2017-01-01", "%Y-%m-%d")
end_date = date.today()

raw_df = pd.DataFrame(columns=tickers)
for ticker in tickers:
    ticker_df = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    raw_df[ticker] = ticker_df["Adj Close"]
    
print(raw_df.head(2))
print(raw_df.tail(2))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                 CL=F   NG=F    RB=F
Date                                
2017-01-03  52.330002  3.327  1.6218
2017-01-04  53.259998  3.267  1.6459
                 CL=F   NG=F    RB=F
Date                                
2023-02-02  75.879997  2.456  2.4523
2023-02-03  73.389999  2.410  2.3210


In [6]:
critical_values = {
    0: {0.9: 13.4294, 0.95: 15.4943, 0.99: 19.9349},  # Critical values for 0 cointegration relationships.
    1: {0.9: 2.7055, 0.95: 3.8415, 0.99: 6.6349},     # Critical values for 1 cointegration relationship.
}

trace0_cv_95 = critical_values[0][0.95]
trace1_cv_95 = critical_values[1][0.95]

# (n, 2) <- (n,) and (n,)
# prices = np.vstack([x, y]).T
prices = raw_df.values

# Vector Autoregressive Model
var = VAR(prices)
lags = var.select_order()
k_ar_diff = lags.selected_orders["aic"]

cj = coint_johansen(prices, det_order=0, k_ar_diff=k_ar_diff)

In [8]:
cj.lr1

array([43.14904592, 17.36060894,  2.3592809 ])

In [10]:
cj.lr1 > trace0_cv_95

array([ True,  True, False])

In [11]:
cj.lr1 > trace1_cv_95

array([ True,  True, False])

In [13]:
(cj.lr1 > trace0_cv_95) and (cj.lr1 > trace1_cv_95)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()